# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [1]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd

conn = sqlite3.connect('data.sqlite')

In [ ]:
# Your code here; create the connection

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [4]:
# Your code here
pd.read_sql("""SELECT customerNumber, contactLastName, contactFirstName
FROM customers
WHERE customerNumber in (SELECT customerNumber FROM orders
WHERE orderDate = '2003-01-31')""", conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


In [5]:
pd.read_sql("""SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;""", conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [18]:
# Your code here
pd.read_sql("""SELECT productCode, SUM(o.quantityOrdered) as product_sum_sold, COUNT(orderNumber)
FROM orderdetails as o
JOIN products as p
USING(productCode)
GROUP BY productName
ORDER BY product_sum_sold""", conn)

,productCode,product_sum_sold,COUNT(orderNumber)
0,S18_4933,767,24
1,S24_1046,803,25
2,S24_3969,824,25
3,S18_2248,832,25
4,S18_2870,855,25
...,...,...,...
104,S50_1341,1074,28
105,S18_3856,1076,28
106,S700_4002,1085,28
107,S18_1342,1111,28


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [21]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
pd.read_sql("""SELECT productName, COUNT(DISTINCT customerNumber) as num_people
FROM products
JOIN orderdetails
USING(productCode)
JOIN orders
USING(orderNumber)
GROUP BY productCode
ORDER BY num_people DESC""",conn)

,productName,num_people
0,1992 Ferrari 360 Spider red,40
1,Boeing X-32A JSF,27
2,1934 Ford V8 Coupe,27
3,1972 Alfa Romeo GTA,27
4,1952 Alpine Renault 1300,27
...,...,...
104,1958 Chevy Corvette Limited Edition,19
105,2002 Chevy Corvette,18
106,1969 Chevrolet Camaro Z28,18
107,1952 Citroen-15CV,18


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [24]:
# Your code here
pd.read_sql("""SELECT employeeNumber, firstName, lastName, o.city, o.officeCode, COUNT(DISTINCT customerNumber) as num_people
FROM offices as o
JOIN employees as e
USING(officeCode)
JOIN customers as c
ON c.salesRepEmployeeNumber = e.employeeNumber
JOIN orders
USING(customerNumber)
JOIN orderdetails
USING(orderNumber)
JOIN products
USING(productCode)
GROUP BY employeeNumber
HAVING num_people < 20 """, conn)

,employeeNumber,firstName,lastName,city,officeCode,num_people
0,1165,Leslie,Jennings,San Francisco,1,6
1,1166,Leslie,Thompson,San Francisco,1,6
2,1188,Julie,Firrelli,Boston,2,6
3,1216,Steve,Patterson,Boston,2,6
4,1286,Foon Yue,Tseng,NYC,3,6
5,1323,George,Vanauf,NYC,3,8
6,1337,Loui,Bondur,Paris,4,6
7,1370,Gerard,Hernandez,Paris,4,7
8,1401,Pamela,Castillo,Paris,4,10
9,1501,Larry,Bott,London,7,8


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [36]:
# Your code here
pd.read_sql("""SELECT employeeNumber, firstName, lastName, COUNT(DISTINCT customerNumber) as num_customers, AVG(creditLimit)
FROM employees
JOIN customers
    ON employees.employeeNumber == customers.salesRepEmployeeNumber
GROUP BY employeeNumber
HAVING AVG(creditLimit) > 15000""", conn)

,employeeNumber,firstName,lastName,num_customers,AVG(creditLimit)
0,1165,Leslie,Jennings,6,100433.333333
1,1166,Leslie,Thompson,6,65266.666667
2,1188,Julie,Firrelli,6,73916.666667
3,1216,Steve,Patterson,6,81533.333333
4,1286,Foon Yue,Tseng,7,66614.285714
5,1323,George,Vanauf,8,80887.500000
6,1337,Loui,Bondur,6,86233.333333
7,1370,Gerard,Hernandez,7,91785.714286
8,1401,Pamela,Castillo,10,81340.000000
9,1501,Larry,Bott,8,91187.500000


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!